# Operaciones CRUD en Neo4j

In [1]:
%load_ext cypher
%matplotlib inline

In [2]:
!curl -s http://localhost:7474/db/data/ | grep neo4j_version

  "neo4j_version" : "3.1.1"


## Borrado de todos los nodos y relaciones de la base de datos

In [3]:
%%cypher
MATCH (n) DETACH DELETE n

0 rows affected.


[]

## Crear 1 nodo

In [4]:
%%cypher
CREATE (ann:Person { name: 'Ann' }) 
RETURN ann

1 labels added.
1 nodes created.
1 properties set.


ann
{u'name': u'Ann'}


In [5]:
%cypher CREATE (dan:Person { name: 'Dan' }) RETURN dan

1 labels added.
1 nodes created.
1 properties set.


dan
{u'name': u'Dan'}


## Crear relaciones

In [6]:
%%cypher
MATCH (a:Person { name: 'Ann' }), 
    (b:Person { name: 'Dan' })
CREATE (a)-[:KNOWS]->(b)

1 relationships created.


[]

## Crear un nodo con MERGE

Sólo se crea si no existe previamente

In [7]:
%%cypher
MERGE (n:Person { name: 'Victor' , age : 20}) RETURN n

1 labels added.
1 nodes created.
2 properties set.


n
"{u'age': 20, u'name': u'Victor'}"


## Crear una relación con MERGE

In [8]:
%%cypher
MATCH (a:Person { name: 'Ann' })
MATCH (b:Person { name: 'Dan' })
MERGE (a)-[r:KNOWS]->(b)
RETURN a,type(r),b

1 rows affected.


a,type(r),b
{u'name': u'Ann'},KNOWS,{u'name': u'Dan'}


In [9]:
%%cypher
MATCH (a:Person { name: 'Ann' }), (b:Person { name: 'Victor' })
MERGE (a)-[r:KNOWS]->(b)
RETURN a,type(r),b

1 relationships created.


a,type(r),b
{u'name': u'Ann'},KNOWS,"{u'age': 20, u'name': u'Victor'}"


## Nodos y relaciones al mismo tiempo

In [10]:
%%cypher
CREATE (Felix:Person {name:'Felix'})
CREATE (Mariam:Person {name:'Mariam',
                       born:1964})
CREATE
  (Felix)-[:KNOWS {since:2010}]->(Mariam)
;

2 labels added.
2 nodes created.
4 properties set.
1 relationships created.


[]

## Encontrar patrones

In [11]:
%%cypher
MATCH (n:Person { name: 'Ann' }) 
RETURN n

1 rows affected.


n
{u'name': u'Ann'}


In [12]:
%%cypher 
MATCH (n:Person) RETURN n

5 rows affected.


n
{u'name': u'Ann'}
{u'name': u'Dan'}
"{u'age': 20, u'name': u'Victor'}"
{u'name': u'Felix'}
"{u'born': 1964, u'name': u'Mariam'}"


In [13]:
%%cypher 
MATCH (n:Person) 
WHERE n.age > 10
RETURN n

1 rows affected.


n
"{u'age': 20, u'name': u'Victor'}"


In [14]:
%%cypher 
MATCH (n:Person { name: 'Ann' })-[r:KNOWS]-(neighbors) 
RETURN n,neighbors

2 rows affected.


n,neighbors
{u'name': u'Ann'},"{u'age': 20, u'name': u'Victor'}"
{u'name': u'Ann'},{u'name': u'Dan'}


In [15]:
%%cypher
MATCH (h:Person)<-[:KNOWS]-(:Person)-[:KNOWS]->(Person) 
RETURN h.name AS Name, COUNT(*) AS count
ORDER BY count DESC
LIMIT 10

2 rows affected.


Name,count
Victor,1
Dan,1


In [16]:
%%cypher
MATCH (n:Person { name: 'Victor' })-[:KNOWS*1..3]-(neighbors) 
RETURN DISTINCT n, neighbors

2 rows affected.


n,neighbors
"{u'age': 20, u'name': u'Victor'}",{u'name': u'Ann'}
"{u'age': 20, u'name': u'Victor'}",{u'name': u'Dan'}


In [17]:
%%cypher
MATCH (n:Person { name: 'Dan' })-[:KNOWS*1..3]-(neighbors) 
RETURN n, collect(DISTINCT neighbors)

1 rows affected.


n,collect(DISTINCT neighbors)
{u'name': u'Dan'},"[{u'name': u'Ann'}, {u'age': 20, u'name': u'Victor'}]"


## Actualizar 1 nodo

In [18]:
%%cypher
MATCH (n:Person {name : "Ann"})
SET n.hair = "Brown"

1 properties set.


[]

## Borrar 1 nodo

In [19]:
%%cypher
CREATE (n:Person {name : "Alex"})
RETURN n;

1 labels added.
1 nodes created.
1 properties set.


n
{u'name': u'Alex'}


In [20]:
%%cypher
MATCH (Alex:Person {name:"Alex"})
DELETE Alex

1 nodes deleted.


[]

## Algoritmos

In [21]:
%%cypher
MATCH p=shortestPath(
  (a:Person { name: 'Ann' })-[:KNOWS]-(b:Person { name: 'Dan'})
)
RETURN p

1 rows affected.


p
"[{u'hair': u'Brown', u'name': u'Ann'}, {}, {u'name': u'Dan'}]"


In [22]:
%%cypher
MATCH p=shortestPath(
  (a:Person { name: 'Victor' })-[:KNOWS*1..3]-(b:Person { name: 'Dan'})
)
RETURN p

1 rows affected.


p
"[{u'age': 20, u'name': u'Victor'}, {}, {u'hair': u'Brown', u'name': u'Ann'}, {}, {u'name': u'Dan'}]"
